In [1]:
import sys
import os

# Get the current directory
current_directory = os.getcwd()

# Add the current directory to the Python path
sys.path.append(current_directory)
print(current_directory)
from spectraltools import Specimen_Collection, Spectrum, create_path_if_not_exists
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math
import re
import scipy


C:\Users\esteb\cicima\escarabajos\gamma


In [2]:
"""This section allows the user to choose their workplace location.
This is important if the user has multiple locations and operating systems in which this 
script is run"""

#select location
working_at = "wfh"

#Training data is used when we are already certain of species and genera for a particular sample
training_data_is_used = False

if working_at == "colaboratory":
  from google.colab import drive
  drive.mount("/content/drive")
  #base folder
  """Select the location for your base folder"""
    
  base_folder = r"/content/drive/My Drive/CICIMA/escarabajos_files"
  
elif working_at == "wfh":

    """Select the location of your base folder"""
    base_folder = r"C:\Users\esteb\cicima\escarabajos"

elif working_at == "cicima_desktop":
  
    """Select the location of your base folder"""
    base_folder = r"C:\Users\EstebanSoto\Jupyter\escarabajos"

elif working_at == "cicima_laptop":
    
    """Select the location of your base folder"""
    base_folder = r"/home/vinicio/escarabajos"

#define the location of the tables with information about the collections and its parent directory

collection_tables_main_path =  os.path.join(base_folder, "L1050_data","collections")
collection_files_main_path = os.path.join(base_folder, "L1050_data")

# Define report location
report_location = os.path.join(base_folder, "reports","data_analysis")

#collection_descriptor = r"CICIMAUCR and ANGSOL" tododelete

#File location and metadata location for collection 1
angsol_collection_path = os.path.join(collection_files_main_path,"ANGSOL","average") 
angsol_collection_metadata = os.path.join(collection_tables_main_path,"CICIMA-beetles-general-inventory - ANGSOL.txt") 

#File location and metadata location for collection 2
cicimaucr_collection_path = os.path.join(collection_files_main_path,r"TRA_data_CICIMA_INBUCR","CICIMAUCR","reflectance")  #listo
cicimaucr_collection_2_path = os.path.join(collection_files_main_path,r"CICIMA-2024-01-REFLECTANCE","average")
cicimaucr_collection_3_path = os.path.join(collection_files_main_path,r"CICIMA-2024-03-REFLECTANCE","without iris nor lens","average")
cicima_ucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - CICIMAUCR.txt") 

#File location and metadata location for collection 3
inbucr_collection_path = os.path.join(collection_files_main_path,r"INBUCR","average") #listo
inbucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - INBUCR.txt") 

#File location and metadata location for collection 4
bioucr_collection_path = os.path.join(collection_files_main_path,r"BIOUCR","average")  #listo
bioucr_metadata = os.path.join(collection_tables_main_path,r"CICIMA-beetles-general-inventory - BIOUCR.txt") 

#agregated data location, here averages and std will be saved when training data and retreived when classifying spectra
agregated_data_location = os.path.join(base_folder, "agregated_data")
agregated_data_avg_path = os.path.join(agregated_data_location,"peak_averages_krc.txt" )  #listo
agregated_data_std_path  = os.path.join( agregated_data_location , r"peak_std_krc.txt") #listo




In [3]:
#Collections
angsol_collection = Specimen_Collection("ANGSOL", angsol_collection_path, angsol_collection_metadata, "HIGH")
angsol_collection.set_description("ANGSOL collection has specimens that belong to Angel Solís. The confidence that we have about specimen identification is high.")

cicimaucr_collection = Specimen_Collection("CICIMAUCR1", cicimaucr_collection_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_2 = Specimen_Collection("CICIMAUCR2", cicimaucr_collection_2_path, cicima_ucr_metadata, "HIGH")
cicimaucr_collection_3 = Specimen_Collection("CICIMAUCR3", cicimaucr_collection_3_path, cicima_ucr_metadata, "HIGH")
inbucr_collection = Specimen_Collection("INBUCR", inbucr_collection_path, inbucr_metadata, "MID")
bioucr_collection = Specimen_Collection("BIOUCR", bioucr_collection_path, bioucr_metadata, "LOW")

collection_list = [
                    angsol_collection,
                    cicimaucr_collection,
                    cicimaucr_collection_2,
                    cicimaucr_collection_3,
                    inbucr_collection,
                    #bioucr_collection,
                    ]
collection_names_set = set([collection.name for collection in collection_list])
collection_names = " ".join( sorted(collection_names_set))
print(collection_names)
#date
from datetime import datetime
current_date = datetime.now().date()

ANGSOL CICIMAUCR1 CICIMAUCR2 CICIMAUCR3 INBUCR


In [4]:
def get_filtered_spectra(collection_list):

    all_spectra = []
    
    for collection in collection_list:
        all_spectra += collection.get_spectra()

    all_spectra = [item for item in all_spectra if item.get_species() in ["kalinini", "resplendens", "cupreomarginata"]]
    return all_spectra
    
all_spectra = get_filtered_spectra(collection_list)

for spectrum in all_spectra:
    print(spectrum.get_species())

resplendens
kalinini
cupreomarginata
kalinini
resplendens
kalinini
cupreomarginata
kalinini
kalinini
kalinini
kalinini
cupreomarginata
resplendens
cupreomarginata
resplendens
cupreomarginata
cupreomarginata
resplendens
resplendens
resplendens
resplendens
kalinini
cupreomarginata
cupreomarginata
resplendens
resplendens
resplendens
resplendens
kalinini
kalinini
kalinini
cupreomarginata
resplendens
cupreomarginata


In [5]:
class Metric():
    """This is an abstract class that represents every metric, allows it to be compared, have a description and a name.
    This is useful when using it in the report methods """
    name = "Metric"
    
    def get_metric_value(self, spectrum):
        metric_value = None
        return metric_value

    def __init__(self, spectrum):
        
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)
        
    @classmethod
    def get_name(cls):
        return cls.name

    @staticmethod
    def description():
        return "No description yet"

    def __lt__(self, other):
        return self.metric_value < other.metric_value

    def __repr__(self):
        return f'{self.name} value: {self.metric_value:.4f} for {self.spectrum.genus} {self.spectrum.species}. File: {self.spectrum.filename}'

In [6]:
class Gamma_Arbitrary_Limits(Metric):
    """This gamma metric calculates the ratio between the maximum in the IR range and the maximum in the visible range. Ranges are static."""
    uv_vis_min_wavelength, uv_vis_max_wavelength = 250.00, 1000.00
    ir_min_wavelength = uv_vis_max_wavelength
    ir_max_wavelength = 2500.00
        
    def get_metric_value(self, spectrum):
        def get_maximum_in_range(spectrum, min_wavelength, max_wavelength):
            measuring_mode = spectrum.metadata["measuring_mode"]
            df = spectrum.data
            max_value = df[(df["wavelength"] > min_wavelength) & (df["wavelength"]  < max_wavelength) ].max()
            #print(f"max value \n {max_value}")
            wavelength, measure = max_value["wavelength"], max_value[measuring_mode]
            return wavelength, measure

        uv_vis_wavelength, uv_vis_max = get_maximum_in_range(spectrum, Gamma_Arbitrary_Limits.uv_vis_min_wavelength, Gamma_Arbitrary_Limits.uv_vis_max_wavelength)
        ir_wavelength, ir_max = get_maximum_in_range(spectrum, Gamma_Arbitrary_Limits.ir_min_wavelength, Gamma_Arbitrary_Limits.ir_max_wavelength)
        metric_value = (uv_vis_max / ir_max)*1.00
        
        return metric_value

    def __init__(self, spectrum):
        self.name = "Gamma_Arbitrary_Limits"
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)

    
    @staticmethod
    def description():
        return f"""This algorithm calculates the ratio between the highest reflectance peak in the visible range (Between {Gamma_Arbitrary_Limits.uv_vis_min_wavelength} nm and {Gamma_Arbitrary_Limits.uv_vis_max_wavelength} nm)
                and the maximum peak in the IR range up to {Gamma_Arbitrary_Limits.ir_max_wavelength} nm. Beyond {Gamma_Arbitrary_Limits.ir_max_wavelength} nm the internal structure's reflectance generates unwanted noise."""
    def __repr__(self):
        return f'Gamma arbitrary limits, value: {self.metric_value:.4f} for {self.spectrum.genus} {self.spectrum.species}. File: {self.spectrum.filename}'

In [7]:
for spectrum in all_spectra:
    print(Gamma_Arbitrary_Limits(spectrum))

Gamma arbitrary limits, value: 1.1390 for Chrysina resplendens. File: ANGSOL0011-3.Sample.ASC
Gamma arbitrary limits, value: 1.3410 for Chrysina kalinini. File: ANGSOL0012-3.Sample.ASC
Gamma arbitrary limits, value: 1.0165 for Chrysina cupreomarginata. File: ANGSOL0013-3.Sample.ASC
Gamma arbitrary limits, value: 1.1098 for Chrysina kalinini. File: ANGSOL0017-3.Sample.ASC
Gamma arbitrary limits, value: 2.0756 for Chrysina resplendens. File: C.RESP.MV23-TX3.ASC
Gamma arbitrary limits, value: 1.7943 for Chrysina kalinini. File: C.KALI.LA22-TX3.ASC
Gamma arbitrary limits, value: 2.7720 for Chrysina cupreomarginata. File: C.CUPR.MV23-TX3.ASC
Gamma arbitrary limits, value: 1.3015 for Chrysina kalinini. File: CICIMAUCR0001-5.Sample.ASC
Gamma arbitrary limits, value: 1.1897 for Chrysina kalinini. File: CICIMAUCR0006-3.Sample.ASC
Gamma arbitrary limits, value: 1.4107 for Chrysina kalinini. File: CICIMAUCR0008-5.Sample.ASC
Gamma arbitrary limits, value: 1.2967 for Chrysina kalinini. File: CICIMA

In [8]:
class Gamma_First_Two_Peaks(Metric):
    """This gamma metric calculates the ratio between the second and first peak."""
    name = "Gamma_First_Two_Peaks"
    
    def get_metric_value(self, spectrum):
        #get list of maxima and minima
        max_i, max_x, max_y = spectrum.get_maxima()
        #Divide second peak over first peak
        metric_value = max_y[1]/max_y[0]
        return metric_value

    def __init__(self, spectrum):
        
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)

    @staticmethod
    def description():
        return f"""This algorithm calculates the ratio between the second and first reflectance peak."""

    def __repr__(self):
        return f'Gamma first two peaks {self.metric_value:.4f} for {self.spectrum.genus} {self.spectrum.species} in {self.spectrum.filename}'

In [9]:
for spectrum in all_spectra:
    print(Gamma_First_Two_Peaks(spectrum))

Gamma first two peaks 0.9297 for Chrysina resplendens in ANGSOL0011-3.Sample.ASC
Gamma first two peaks 1.5176 for Chrysina kalinini in ANGSOL0012-3.Sample.ASC
Gamma first two peaks 1.1950 for Chrysina cupreomarginata in ANGSOL0013-3.Sample.ASC
Gamma first two peaks 1.3003 for Chrysina kalinini in ANGSOL0017-3.Sample.ASC
Gamma first two peaks 1.0613 for Chrysina resplendens in C.RESP.MV23-TX3.ASC
Gamma first two peaks 1.2895 for Chrysina kalinini in C.KALI.LA22-TX3.ASC
Gamma first two peaks 0.8563 for Chrysina cupreomarginata in C.CUPR.MV23-TX3.ASC
Gamma first two peaks 1.3487 for Chrysina kalinini in CICIMAUCR0001-5.Sample.ASC
Gamma first two peaks 1.6206 for Chrysina kalinini in CICIMAUCR0006-3.Sample.ASC
Gamma first two peaks 1.2985 for Chrysina kalinini in CICIMAUCR0008-5.Sample.ASC
Gamma first two peaks 1.2140 for Chrysina kalinini in CICIMAUCR0009-3.Sample.ASC
Gamma first two peaks 1.0668 for Chrysina cupreomarginata in CICIMAUCR0012-3.Sample.ASC
Gamma first two peaks 0.8373 for C

In [10]:
class Gamma_Area_Under_Curve_Naive(Metric):
    #this is a subclass of Gamma
    #get_gamma_factor must be redefined
    visible_start_wavelength = 450
    visible_end_wavelength = ir_start_wavelength = 800
    ir_end_wavelength = 1500
    name = "Gamma_Area_Under_Curve_Naive"
    
    def __init__(self, spectrum):
        
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)
        
   
    def description():
        return f"""This method calculates the ratio between the area under the curve for the spectrum between {Gamma_Area_Under_Curve_Naive.visible_start_wavelength} 
        and {Gamma_Area_Under_Curve_Naive.visible_end_wavelength} nm (visible range) and between {Gamma_Area_Under_Curve_Naive.ir_start_wavelength} nm and 
        {GammaAreaUnderCurveNaive.ir_end_wavelength} nm (infrared range)."""


    def get_metric_value(self, spectrum):

        def get_area_under_curve(spectrum, start_wavelength, finish_wavelength):
            # Subset the DataFrame to the range of interest
            subset_df = df[(df['wavelength'] >= start_wavelength) & (df['wavelength'] <= finish_wavelength)]

            # Extract the wavelengths and heights as arrays
            wavelengths = subset_df['wavelength'].values
            heights = subset_df[spectrum.measuring_mode].values

            # Calculate the area under the curve using the trapezoidal rule
            area_under_curve = np.trapz(heights, wavelengths)

            #print("Area under the curve:", area_under_curve)
            return area_under_curve

        import numpy as np
        df = spectrum.get_normalized_spectrum()
        area_uv_visible = get_area_under_curve(spectrum, Gamma_Area_Under_Curve_Naive.visible_start_wavelength, Gamma_Area_Under_Curve_Naive.visible_end_wavelength)
        area_ir = get_area_under_curve(spectrum, Gamma_Area_Under_Curve_Naive.ir_start_wavelength, Gamma_Area_Under_Curve_Naive.ir_end_wavelength)
        metric = area_ir/area_uv_visible
        return metric


In [11]:
for spectrum in all_spectra:
    print(Gamma_Area_Under_Curve_Naive(spectrum))

Gamma_Area_Under_Curve_Naive value: 2.4134 for Chrysina resplendens. File: ANGSOL0011-3.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 2.3558 for Chrysina kalinini. File: ANGSOL0012-3.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 2.9187 for Chrysina cupreomarginata. File: ANGSOL0013-3.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 2.5427 for Chrysina kalinini. File: ANGSOL0017-3.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 1.2567 for Chrysina resplendens. File: C.RESP.MV23-TX3.ASC
Gamma_Area_Under_Curve_Naive value: 1.5541 for Chrysina kalinini. File: C.KALI.LA22-TX3.ASC
Gamma_Area_Under_Curve_Naive value: 1.0513 for Chrysina cupreomarginata. File: C.CUPR.MV23-TX3.ASC
Gamma_Area_Under_Curve_Naive value: 2.0895 for Chrysina kalinini. File: CICIMAUCR0001-5.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 2.1405 for Chrysina kalinini. File: CICIMAUCR0006-3.Sample.ASC
Gamma_Area_Under_Curve_Naive value: 1.5817 for Chrysina kalinini. File: CICIMAUCR0008-5.Sample.ASC
Gamma_Area_Under_Curve_

In [12]:
class  Gamma_Area_Under_Curve_First_Min_Cut(Metric):
    #this is a subclass of Gamma
    #get_gamma_factor must be redefined
    visible_range_start_wavelength = 450
    name = "gamma_area_under_curve_cut_first_minimum"

    def __init__(self, spectrum):
        
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)
        
    def description():
        return f"""This algorithm calculates the area for the visible region (starting at {Gamma_Area_First_Min_Cut.visible_range_start_wavelength} 
        and ending in the first minima between the maximum in the visible range and the maximum in the IR range. 
        Then calculates the area of the IR range up to the second minumum. The ratio between these two areas is the gamma value."""

    def get_metric_value(self, spectrum):

        def get_area_under_curve(spectrum, start_wavelength, finish_wavelength):
            # Assuming your DataFrame is named df and has columns 'wavelength' and 'height'
            # Let's say you have start_wavelength and finish_wavelength variables for the range you want to integrate over
            # Subset the DataFrame to the range of interest
            subset_df = df[(df['wavelength'] >= start_wavelength) & (df['wavelength'] <= finish_wavelength)]

            # Extract the wavelengths and heights as arrays
            wavelengths = subset_df['wavelength'].values
            heights = subset_df[spectrum.measuring_mode].values

            # Calculate the area under the curve using the trapezoidal rule
            area_under_curve = np.trapz(heights, wavelengths)

            # print("Area under the curve:", area_under_curve)
            return area_under_curve

        import numpy as np

        #test_spectrum = filtered_spectra[0]
        #get the highest data recorded
        max_value = spectrum.data[spectrum.measuring_mode].max()
        #get maxima and minima
        x = spectrum.data["wavelength"].values
        y = spectrum.data[spectrum.measuring_mode].values

        #get x and y positions of maxima and minima
        max_i, max_xs, max_ys = spectrum.get_maxima()
        min_i, min_xs, min_ys= spectrum.get_minima()

        #get x locations of first and second maxima and the minimum in between
        first_max_x = max_xs[0]
        try:
            second_max_x = max_xs[1]
        except Exception as e:
            second_max_x = x.max()
            print(e)
        try:
            second_max_y = max_ys[1]
        except Exception as e:
            second_max_y = 0
            print(e)

        min_in_between_i = 0
        min_in_between_x =0
        min_in_between_y =0
        #get the location of the minimum in between
        for index in min_i:
            #print("index")
            if first_max_x <= x[index] <= second_max_x:
                min_in_between_i = index
                min_in_between_x = x[index]
                min_in_between_y = y[index]
                break

        # print(f"min in bet: {min_in_between_i} {min_in_between_x} {min_in_between_y} ")
        #second minimum
        #get the location of the second minimum
        min_after_second_max_i = 0
        min_after_second_max_x = 0
        min_after_second_max_y = 0
        for index in min_i:
            # print(f" second_max_x  <= x[index] { second_max_x  <= x[index]}")
            if second_max_x  <= x[index]:
                min_after_second_max_i = index
                min_after_second_max_x = x[index]
                min_after_second_max_y = y[index]
                break


        # print(f"min after: {min_after_second_max_i} {min_after_second_max_x} {min_after_second_max_y} ")

        x_values = [first_max_x, min_in_between_x, second_max_x, min_after_second_max_x]
        y_values = [max_ys[0]/max_value, min_in_between_y/max_value, second_max_y/max_value, min_after_second_max_y/max_value]
        #get the normalized spectrum
        df = spectrum.get_normalized_spectrum()
        #plot
        x = df["wavelength"].values
        y =df[spectrum.measuring_mode].values

        #modify y to have last value equal to first one
        y_mod = y
        y_mod[-1] = y_mod[0]
        
        #split x, y LEFT
        #print(f"fmi: {min_in_between_i}")
        x_left = x[:min_in_between_i]
        y_left = y[:min_in_between_i]
        #set last one to zero for picture to be displaye properly
        y_left[-1] = y_left[0]
        

        #split x, y RIGHT
        #print(f"min_after_second_max_i: {min_after_second_max_i}")
        x_right = x[min_in_between_i:min_after_second_max_i]
        y_right = y[min_in_between_i:min_after_second_max_i]
        #set last one to zero for picture to be displaye properly
        y_right[0] = y_right[-1] = y_left[0]
        
        #show figure

        area_uv_visible = get_area_under_curve(spectrum, Gamma_Area_Under_Curve_First_Min_Cut.visible_range_start_wavelength, min_in_between_x)
        area_ir = get_area_under_curve(spectrum, min_in_between_x, min_after_second_max_x)
        gamma = area_ir/area_uv_visible
        #print(f"gamma: {gamma}")
        return gamma


In [13]:
for spectrum in all_spectra:
    print(Gamma_Area_Under_Curve_First_Min_Cut(spectrum))

gamma_area_under_curve_cut_first_minimum value: 1.9944 for Chrysina resplendens. File: ANGSOL0011-3.Sample.ASC
gamma_area_under_curve_cut_first_minimum value: 2.9641 for Chrysina kalinini. File: ANGSOL0012-3.Sample.ASC
gamma_area_under_curve_cut_first_minimum value: 3.5576 for Chrysina cupreomarginata. File: ANGSOL0013-3.Sample.ASC
gamma_area_under_curve_cut_first_minimum value: 3.2443 for Chrysina kalinini. File: ANGSOL0017-3.Sample.ASC
gamma_area_under_curve_cut_first_minimum value: 1.8487 for Chrysina resplendens. File: C.RESP.MV23-TX3.ASC
gamma_area_under_curve_cut_first_minimum value: 2.3989 for Chrysina kalinini. File: C.KALI.LA22-TX3.ASC
gamma_area_under_curve_cut_first_minimum value: 2.0651 for Chrysina cupreomarginata. File: C.CUPR.MV23-TX3.ASC
gamma_area_under_curve_cut_first_minimum value: 2.7227 for Chrysina kalinini. File: CICIMAUCR0001-5.Sample.ASC
gamma_area_under_curve_cut_first_minimum value: 3.7340 for Chrysina kalinini. File: CICIMAUCR0006-3.Sample.ASC
gamma_area_und

In [14]:
class Gamma_Vector_Relative_Reflectance(Metric):
    """This gamma metric calculates a vector with all the relative heights with respect to the first peak."""
    name = "Gamma_Vector_Relative_Reflectance"
    
    def get_metric_value(self, spectrum):
        #get list of maxima and minima
        max_i, max_x, max_y = spectrum.get_maxima()
        
        #Divide every peak over first peak
        metric_value = max_y/max_y[0]
        return metric_value

    def __init__(self, spectrum):
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)

    @staticmethod
    def description():
        return f"""This gamma metric calculates a vector with all the relative heights with respect to the first peak"""

    def __repr__(self):
        return f'Gamma vector relative reflectance: {self.metric_value} for {self.spectrum.genus} {self.spectrum.species} in {self.spectrum.filename}'

In [15]:
for spectrum in all_spectra:
   print(Gamma_Vector_Relative_Reflectance(spectrum))

Gamma vector relative reflectance: [1.         0.92966156 0.87795186 0.84244486 0.57505403 0.5182256 ] for Chrysina resplendens in ANGSOL0011-3.Sample.ASC
Gamma vector relative reflectance: [1.         1.51757981 1.11461812 0.81330654 0.73627098] for Chrysina kalinini in ANGSOL0012-3.Sample.ASC
Gamma vector relative reflectance: [1.         1.19502384 1.17567787 1.13509447 0.81120859 0.73377343] for Chrysina cupreomarginata in ANGSOL0013-3.Sample.ASC
Gamma vector relative reflectance: [1.         1.30028523 1.17160619 1.17018845 0.76076238 0.66803659] for Chrysina kalinini in ANGSOL0017-3.Sample.ASC
Gamma vector relative reflectance: [1.         1.06126356 0.44042747] for Chrysina resplendens in C.RESP.MV23-TX3.ASC
Gamma vector relative reflectance: [1.         1.28946201 0.52917791 0.43903273] for Chrysina kalinini in C.KALI.LA22-TX3.ASC
Gamma vector relative reflectance: [1.         0.8562896  0.35553578 0.34434818] for Chrysina cupreomarginata in C.CUPR.MV23-TX3.ASC
Gamma vector rel

In [16]:
class Wavelength_Vector(Metric):
    """This gamma metric calculates a vector with each peak's wavelength."""
    name = "Wavelength_Vector"
    
    def get_metric_value(self, spectrum):
        #get list of maxima and minima
        max_i, max_x, max_y = spectrum.get_maxima()
        
        #Divide every peak over first peak
        metric_value = max_x
        return metric_value

    def __init__(self, spectrum):
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)

    @staticmethod
    def description():
        return f"""This metric calculates a vector with each peak's wavelength."""

    def __repr__(self):
        return f'Vector wavelength : {self.metric_value} for {self.spectrum.genus} {self.spectrum.species} in {self.spectrum.filename}'

In [17]:
for spectrum in all_spectra:
   print(Wavelength_Vector(spectrum))

Vector wavelength : [ 614.  830. 1126. 1308. 1659. 1855.] for Chrysina resplendens in ANGSOL0011-3.Sample.ASC
Vector wavelength : [ 587.  802. 1316. 1653. 1855.] for Chrysina kalinini in ANGSOL0012-3.Sample.ASC
Vector wavelength : [ 607.  934. 1130. 1292. 1656. 1816.] for Chrysina cupreomarginata in ANGSOL0013-3.Sample.ASC
Vector wavelength : [ 578.  815. 1134. 1319. 1656. 1862.] for Chrysina kalinini in ANGSOL0017-3.Sample.ASC
Vector wavelength : [ 586.  755. 1385.] for Chrysina resplendens in C.RESP.MV23-TX3.ASC
Vector wavelength : [ 586.  776. 1396. 1668.] for Chrysina kalinini in C.KALI.LA22-TX3.ASC
Vector wavelength : [ 572.  742. 1113. 1213.] for Chrysina cupreomarginata in C.CUPR.MV23-TX3.ASC
Vector wavelength : [ 567.  765. 1123. 1300. 1649. 1848.] for Chrysina kalinini in CICIMAUCR0001-5.Sample.ASC
Vector wavelength : [ 550.  772.  858. 1119. 1303. 1654. 1858.] for Chrysina kalinini in CICIMAUCR0006-3.Sample.ASC
Vector wavelength : [ 573.  742.  858. 1118. 1304.] for Chrysina 

In [23]:
class Critical_Points(Metric):
    """This metric returns a vector with each critical point wavelength and relative reflectance."""
    name = "Critical_Points"
    
    def get_metric_value(self, spectrum):
       
        #get list of maxima and minima
        #crit_i, crit_x, crit_y = 
        
        peaks = spectrum.get_critical_points()
        metric_value = [list(element) for element in peaks]
        #first maximum is metric_value[0][1]
        first_max = metric_value[0][1]
        metric_value = [obj/first_max for obj in metric_value]
        return metric_value
     

    def __init__(self, spectrum):
        self.spectrum = spectrum
        self.metric_value = self.get_metric_value(spectrum)

    @staticmethod
    def description():
        return f"""This metric returns a vector with each critical point wavelength and relative reflectance."""

    def __repr__(self):
        return f'Critical_Points : {self.metric_value} for {self.spectrum.genus} {self.spectrum.species} in {self.spectrum.filename}'

In [24]:
for spectrum in all_spectra:
   print(Critical_Points(spectrum))

Critical_Points : [array([27.73081204,  1.        ]), array([37.48627686,  0.92966156]), array([50.8548768 ,  0.87795186]), array([59.0747592 ,  0.84244486]), array([74.92738953,  0.57505403]), array([83.77957058,  0.5182256 ]), array([30.7568127 ,  0.59169979]), array([40.01547144,  0.87091375]), array([46.20296534,  0.84356913]), array([54.42284773,  0.80826004]), array([68.28825375,  0.51189711]), array([79.03733073,  0.48556358]), array([88.2056611 ,  0.26375421])] for Chrysina resplendens in ANGSOL0011-3.Sample.ASC
Critical_Points : [array([38.07027669,  1.        ]), array([52.01424516,  1.51757981]), array([85.35005814,  1.11461812]), array([107.20641801,   0.81330654]), array([120.3072628 ,   0.73627098]), array([41.05363739,  0.85427133]), array([57.07298721,  1.19666126]), array([68.94157432,  1.08998782]), array([78.21593474,  1.06563621]), array([97.86720192,  0.7549738 ]), array([113.49741774,   0.70496982]), array([126.72797386,   0.40429894])] for Chrysina kalinini in AN

## Averages

In [ ]:
#Saving_averages is True only with trusty spectra
saving_averages = True
#This class gets the average and standard deviation per species for a particular Gamma class
def get_gamma_avg_and_std(Metric,filtered_spectra):

    #Define metric name
    
    #Calculate gammas
    metric_list = []

    for spectrum in filtered_spectra:
        #print(spectrum.get_normalized_spectrum())
        try:
            metric = Metric(spectrum)
            metric_list.append(metric)
            
        except Exception as e:
            print(e)
    #Order the list
    sorted(metric_list)
    #Create a dataframe
    metric_df = pd.DataFrame(columns=["species", "genus", "gamma", "code", "filename"])

    #add specimen information to the gammas
    for index, metric in enumerate(metric_list):
        metric_df.loc[index,"species"] = metric.spectrum.species
        metric_df.loc[index,"genus"] = metric.spectrum.genus
        metric_df.loc[index,"metric"] = metric.metric_value
        metric_df.loc[index,"code"] = metric.spectrum.code
        metric_df.loc[index,"filename"] = metric.spectrum.filename

    #get info on df
    grouped_stats = metric_df.groupby('species')['metric'].agg(['mean', 'std'])
    grouped_stats = grouped_stats.T
    print(grouped_stats)

    #save info in a file 
    #save information
    #path_location = agregated_data_location + "\\aggregated_data\\gamma"
    path_location = os.path.join(agregated_data_location, "metric_avg_std")
    create_path_if_not_exists(path_location)
    
    path_and_filename = os.path.join( path_location, f'{Metric.get_name()}.txt')
    grouped_stats.to_csv( path_and_filename, index=False, sep = "\t")
    #return path
    return path_and_filename

In [ ]:
get_gamma_avg_and_std(Gamma_Arbitrary_Limits,all_spectra)

In [ ]:
get_gamma_avg_and_std(Gamma_First_Two_Peaks,all_spectra)

In [ ]:
get_gamma_avg_and_std(Gamma_Area_Under_Curve_Naive,all_spectra)

In [ ]:
get_gamma_avg_and_std(Gamma_Area_Under_Curve_First_Min_Cut,all_spectra)

In [ ]:
class Metric_Testbench():
    """This class tests the metrics for the selected spectra and creates a boxplot for the species selected.
    Returns the path to the boxplot image"""
    #Calculate gammas
    
    def __init__(self, Metric, filtered_spectra):
        self.metric_class = Metric
        self.spectra = filtered_spectra
        self.test_df, self.boxplot_path = self.get_boxplot()
        
    def get_boxplot(self):
        
        filtered_spectra = self.spectra
        Metric = self.metric_class
        
        metric_list = []

        for spectrum in filtered_spectra:
            #print(spectrum.get_normalized_spectrum())
            try:
                metric = Metric(spectrum)
                metric_list.append(metric)
            except Exception as e:
                print(e)
        sorted(metric_list)

        metric_df = pd.DataFrame(columns=["species", "genus", "gamma", "code", "filename"])

        #add specimen information to the gammas
        for index, metric in enumerate(metric_list):
            metric_df.loc[index,"species"] = metric.spectrum.species
            metric_df.loc[index,"genus"] = metric.spectrum.genus
            metric_df.loc[index,"metric"] = metric.metric_value
            metric_df.loc[index,"code"] = metric.spectrum.code
            metric_df.loc[index,"filename"] = metric.spectrum.filename

        #print(gamma_df)

        #finally, information is presented as a boxplot and saved
        ax = metric_df.boxplot(column=["metric"], by=["species"], ax=None, fontsize=None, rot=90, grid=True, figsize=(4*3, 4*3), layout=None, return_type=None, backend=None, showfliers=False)
        fig = ax.figure
        plt.title(f" Metric: {Metric.get_name() }. Collections: {collection_names}. \n Metric values for C. resplendens, C. kalinini and C. cupreomarginata.")
        
        path= os.path.join(report_location, "report_images", "gamma_image")
        create_path_if_not_exists(path)
        filename = os.path.join(path, f"{Metric.get_name()} "+ collection_names + f"-{current_date}" +".jpeg") 
        fig.savefig(filename)
        
        return metric_df, filename

In [ ]:
Metric_Testbench(Gamma_Arbitrary_Limits, all_spectra)

In [ ]:
Metric_Testbench(Gamma_First_Two_Peaks, all_spectra)

In [ ]:
Metric_Testbench(Gamma_Area_Under_Curve_Naive, all_spectra)

In [ ]:
Metric_Testbench(Gamma_Area_Under_Curve_First_Min_Cut, all_spectra)